In [29]:
import os
import pandas as pd
import pickle
import altair as alt

In [3]:
weather_path = "fids_data2.pkl"

In [5]:
weather_data = pickle.load(open(weather_path,'rb'))

In [61]:
def get_locations_in_country(df,country,fids_dict):
    not_in = ["American Samoa",
            "Chicago",
            "Diamond Princess cruise ship",
            "Grand Princess",
            "Grand Princess Cruise Ship",
            "Guam",
            "Northern Mariana Islands",
            "Puerto Rico",
            "Recovered",
            "Unassigned Location (From Diamond Princess)",
            "United States Virgin Islands",
            "US",
            "Virgin Islands",
            "Wuhan Evacuee"]
    locations = df[(df['Country/Region']==country)]['Province/State'].unique()
    locations = filter(lambda location:"," not in location,locations)
    locations = [location for location in locations if location not in not_in]
    fids = list(map(fids_dict.get,locations))
    return [f for f in fids if f is not None],dict(zip(fids,locations))
covid_df_head = './covid_data'
covid_dfs = dict(map(lambda p:(os.path.splitext(p)[0],pd.read_csv(os.path.join(covid_df_head,p))),os.listdir(covid_df_head)))
covid_dfs['covid_19_data'][covid_dfs['covid_19_data']['Country/Region']=='US']
fids_dict = {v['name']:v['id'] for k,v in pd.read_csv('./ncei_data/locations.txt').to_dict('index').items()}
fids_locations,location_dict = get_locations_in_country(covid_dfs['covid_19_data'],'US',fids_dict)

In [63]:
weather_data['FIPS:48']['datatype'].unique()

array(['PRCP', 'SNOW', 'SNWD', 'DAPR', 'MDPR', 'WESD', 'WESF'],
      dtype=object)

In [58]:
def clean_weather_dataframe(df,fids,location):
    df = df[df['datatype']=='TAVG'].groupby(['date']).mean()
    df = df.drop([c for c in df.columns if c!='value'],axis=1)
    df = pd.DataFrame(df).rename(columns={'value':'Average Temperature (deg F)'})
    return df

In [60]:
ex_fids = list(weather_data.keys())[0]
ex_fids_data = clean_weather_dataframe(weather_data[ex_fids],ex_fids,location_dict[ex_fids]).reset_index()
alt.Chart(ex_fids_data).mark_point().encode(
    y='Average Temperature (deg F)',
    x='date'
)

alt.Chart(...)

In [75]:
fids_timeseries = {}
#print(weather_data.keys())
for fids in weather_data:
    #print(fids,location_dict[fids])
    fids_timeseries[location_dict[fids]] = clean_weather_dataframe(weather_data[fids],fids,location_dict[fids])
    fids_timeseries[location_dict[fids]]['location']=location_dict[fids]
    fids_timeseries[location_dict[fids]]['fids']=fids
print(len(fids_timeseries))

51


In [76]:
print(fids_timeseries['Texas'])
loc_timeseries = pd.concat([df for df in fids_timeseries.values()])

Empty DataFrame
Columns: [Average Temperature (deg F), location, fids]
Index: []


In [39]:
loc_timeseries

,Average Temperature (deg F),location,fids
date,,,
2020-02-01T00:00:00,38.933333,Washington,FIPS:53
2020-02-08T00:00:00,33.666667,Washington,FIPS:53
2020-02-12T00:00:00,31.333333,Washington,FIPS:53
2020-02-15T00:00:00,33.652174,Washington,FIPS:53
2020-02-16T00:00:00,32.583333,Washington,FIPS:53
...,...,...,...
2020-06-26T00:00:00,74.538462,Alabama,FIPS:01
2020-06-27T00:00:00,77.384615,Alabama,FIPS:01
2020-06-28T00:00:00,81.153846,Alabama,FIPS:01


In [40]:
loc_timeseries.describe()

,Average Temperature (deg F)
count,4428.000000
mean,51.614499
std,16.432709
min,-13.304688
25%,40.252976
50%,52.000000
75%,65.402273
max,84.454545


In [90]:
#the max value doesn't seem correct for Texas in June.. I think we lost some points
set(location_dict.values())-set(loc_timeseries['location'].unique())

{'Arizona',
 'California',
 'Colorado',
 'District of Columbia',
 'Illinois',
 'Iowa',
 'Kansas',
 'Michigan',
 'Missouri',
 'Nebraska',
 'New Mexico',
 'New York',
 'North Carolina',
 'Oregon',
 'Pennsylvania',
 'Texas',
 'Wisconsin'}

In [74]:
#We lost some pretty big data points - California - Illinois - New York - Texas. The 4 states with the four biggest populations
charts = [alt.Chart(loc_timeseries[loc_timeseries['location']==loc].reset_index()
                   ).mark_line().encode(x='date',y='Average Temperature (deg F)') for loc in loc_timeseries['location'].unique()]
alt.layer(*charts)

alt.LayerChart(...)

In [93]:
alt.Chart(loc_timeseries.reset_index()).mark_line().encode(x='date',y='mean(Average Temperature (deg F))').properties(width=400)

alt.Chart(...)

In [94]:
loc_timeseries.to_csv('weather_timeseries.csv')